In [ ]:
import pandas as pd
import torch
from collections import Counter
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [ ]:
data=pd.read_csv('/kaggle/input/news-clickbait-dataset/train1.csv')

In [ ]:
dic={}
for s in data['headline'][:3200]:
  separated_words=s.split()
  separated_words.append('.')
  for w1 ,w2 in zip(separated_words,separated_words[1:]):
    bigram=(w1,w2)
    dic[bigram]=dic.get(bigram,0)+1

In [ ]:
words = set()
for key in dic.keys():
    words.update(key)

words=list(words)
words=sorted(words)
end_token=words.pop(words.index('.'))
words.insert(0,'.')
stoi = {word: index for index, word in enumerate(words)}

In [ ]:
xs,ys=[],[]
dic={}
for s in data['headline'][:3200]:
  separated_words=s.split()
  separated_words.append('.')
  for w1 ,w2 in zip(separated_words,separated_words[1:]):
    ix1=stoi[w1]
    ix2=stoi[w2]
    xs.append(ix1)
    ys.append(ix2)
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [ ]:
print(xs)
print(ys)

In [ ]:
W = torch.randn((len(words),len(words)),requires_grad=True)

In [ ]:
batch=[]
batch_size=32
batch_ans=[]
number_of_batches=xs.numel()/batch_size
for i in range (int(number_of_batches)):
    batch.append(xs[batch_size*i:batch_size*(i+1)][:])
    batch_ans.append(ys[batch_size*i:batch_size*(i+1)][:])
    #if i==int(number_of_batches)-1:
                #  epoch.append(xs[batch_size*(i+1):xs.shape[0]])
                #  epoch_ans.append(ys[batch_size*(i+1):ys.shape[0]])

In [ ]:
for k in range (15):
    print(k)
    batch_enc=[]
    for i in range(len(batch)):
        W.grad=None
        #Forward Propagation
        batch_enc.append(F.one_hot(batch[i],num_classes=len(words)).float().requires_grad_(True))
    
    
        logits=(batch_enc[i] @ W)
        counts=logits.exp()
        prob=counts/counts.sum(1,keepdims=True)
        loss=-prob[torch.arange(32),batch_ans[i]].log().mean()+0.01*(W**2).mean()
     #Backward Propagation
        loss.backward() 
        print(loss.item())
    
    #Updation of weights
        W.data += -1 * W.grad
print(loss.item())

In [ ]:
for i in range(10):
  
  out = []
  ix = 0
  for k in range(5):
    xenc = F.one_hot(torch.tensor([ix]), num_classes=len(words)).float()
    logits = xenc @ W 
    counts = logits.exp() 
    p = counts / counts.sum(1, keepdims=True) 
    
    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(words[ix])
    
    out.append(' ')
   
  print(''.join(out))

In [ ]:
torch.save(W, 'model_weights.pth')